In [4]:
from imports import *
from information_conditions import Information_Conditions
from base_ecopg import *
from helper_functions import *
from simulation_and_results_functions import *

In [49]:
np.set_printoptions(precision=8, suppress=True, linewidth=200)


In [3]:
def add_degraded_state_policies_both_state_and_action(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()

        for i in [0, 2, 4, 6]:
            x = np.random.rand()
            strategy_propserous_and_degraded_state.insert(i, x)
        
        return strategy_propserous_and_degraded_state


def add_degraded_state_policies_only_state(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()
        x = np.random.rand()
        strategy_propserous_and_degraded_state.insert(0, x)
        
        return strategy_propserous_and_degraded_state



In [58]:
def create_policy_from_strategy(agent_1_strategy, agent_2_strategy):


    agent_1_strategy = [[x, 1-x] for x in agent_1_strategy]
    agent_2_strategy = [[x, 1-x] for x in agent_2_strategy]

    policy = np.array([agent_1_strategy, agent_2_strategy])

    return policy


def epsilon_noise_for_strategy(determinstic_strategy, epsilon):
    #to avoid pure determinstic strategies, we remove e from 0  
    strategy_with_epsilon = [x + epsilon if x == 0 else x - epsilon if x == 1 else x for x in determinstic_strategy]
    return strategy_with_epsilon



def create_policy_with_noise_from_deterministic_strategy(agent_1_strategy, agent_2_strategy, epsilon):

    
    determinstic_policy = create_policy_from_strategy(agent_1_strategy, agent_2_strategy)
    
    
    agent_1_strategy_with_noise = epsilon_noise_for_strategy(agent_1_strategy, epsilon)
    agent_2_strategy_with_noise = epsilon_noise_for_strategy(agent_2_strategy, epsilon)

    noisy_policy = create_policy_from_strategy(agent_1_strategy_with_noise, agent_2_strategy_with_noise)

    return noisy_policy, determinstic_policy


In [6]:
def calculate_avg_value_given_policy_obs(policy, mode):

    mae_ecopg =  create_mae_ecopg_for_given_mode_POstratAC(mode)
    # mae_ecopg.has_last_statdist = False
    # mae_ecopg._last_statedist = None

    Vio = mae_ecopg.Vio(policy)
    obsdist = mae_ecopg.obsdist(policy)

    # print(Vio, "Vio")
    # print(obsdist, "obsdist")

    if mode == 'complete' or mode == 'ecological':
        Vio = Vio[:,1::2]
        obsdist = obsdist[:,1::2]
        obsdist = obsdist/obsdist.sum(axis = 1)

    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vio, [agents, states], obsdist , [agents, states], [agents])
    # print(avg_value_agent_1, avg_value_agent_2, "value of agents")

    return avg_value_agent_1, avg_value_agent_2

In [7]:
def calculate_avg_value_given_policy_state_mae_ecopg(policy, mode):

    mae_ecopg =  create_mae_ecopg_for_given_mode_POstratAC(mode)
    Vis = mae_ecopg.Vis(policy)
    # mae_ecopg.has_last_statdist = False
    # mae_ecopg._last_statedist = None
    # state_dist = mae_ecopg._numpyPs(policy)
    
    state_dist = mae_ecopg.Ps(policy)
    
    # E = 0.01
    # state_dist = np.where(state_dist == 0, E, state_dist)

    if mode == 'complete' or mode == 'ecological':
        Vis = Vis[:,1::2]
        state_dist = state_dist[1::2]

    state_dist = state_dist/np.sum(state_dist)

    # print(Vis, "vis")
    # print(state_dist, "state dist")

    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vis, [agents, states], state_dist , [states], [agents])
    # print(avg_value_agent_1, avg_value_agent_2, "value of agents")

    return avg_value_agent_1, avg_value_agent_2



def calculate_avg_value_given_policy_state_test(policy, mode, mae):

    Vis = mae.Vis(policy)
    state_dist = mae.Ps(policy)

    # print(Vis, "vis")
    # print(state_dist, "state dist")
    
    if mode == 'complete' or mode == 'ecological':
        Vis = Vis[:,1::2]
        state_dist = state_dist[1::2]
        state_dist = state_dist/np.sum(state_dist)
    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vis, [agents, states], state_dist , [states], [agents])


    return  avg_value_agent_1, avg_value_agent_2



In [2]:
def create_determinstic_strategies_set_for_both_players(mode):
    '''creates deterministic strategy sets for given mode '''

    # E = 0.05
    mae_ecopg_for_evaluating_no_of_states = create_mae_ecopg_for_given_mode_POstratAC(mode)
    number_of_states = mae_ecopg_for_evaluating_no_of_states.Q
    
    if mode == 'none' or mode == 'social':
        determinstic_strategy_itertools = itertools.product([1 , 0], repeat = number_of_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_full = {str(strat):strat for strat in determinisic_strategy_lists}

    else:
        number_of_prosperous_states = int(number_of_states/2)
        determinstic_strategy_itertools = itertools.product([1, 0], repeat = number_of_prosperous_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_only_prosperous = {str(strat):strat for strat in determinisic_strategy_lists}

        if mode == 'complete':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}
        elif mode == 'ecological':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_only_state(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}


    strategy_set_p1 = all_determinstic_strategy_dictionary_full
    strategy_set_p2 = all_determinstic_strategy_dictionary_full

    return strategy_set_p1, strategy_set_p2

In [23]:
def create_determinstic_strategies_set_for_both_players_with_epsilon(mode, epsilon):
    '''creates deterministic strategy sets for given mode '''

    mae_ecopg_for_evaluating_no_of_states = create_mae_ecopg_for_given_mode_POstratAC(mode)
    number_of_states = mae_ecopg_for_evaluating_no_of_states.Q
    
    if mode == 'none' or mode == 'social':
        determinstic_strategy_itertools = itertools.product([1 - epsilon , 0 + epsilon], repeat = number_of_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_full = {str(strat):strat for strat in determinisic_strategy_lists}

    else:
        number_of_prosperous_states = int(number_of_states/2)
        determinstic_strategy_itertools = itertools.product([1 - epsilon , 0 + epsilon], repeat = number_of_prosperous_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_only_prosperous = {str(strat):strat for strat in determinisic_strategy_lists}

        if mode == 'complete':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}
        elif mode == 'ecological':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_only_state(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}


    strategy_set_p1 = all_determinstic_strategy_dictionary_full
    strategy_set_p2 = all_determinstic_strategy_dictionary_full

    return strategy_set_p1, strategy_set_p2



In [10]:
def reward_matrix_to_excel(p1_reward_matrix, p2_reward_matrix, strategy_set_p1, strategy_set_p2):

    '''p1 rewards pg 1. p2 rewards pg 2. rows -> p1 strategies, cols -> p2 strategies in both cases '''
    p1_reward_matrix = pd.DataFrame(p1_reward_matrix, index = list(strategy_set_p1.keys()), columns= list(strategy_set_p2.keys()))
    p2_reward_matrix = pd.DataFrame(p2_reward_matrix, index = list(strategy_set_p1.keys()), columns= list(strategy_set_p2.keys()))

    with pd.ExcelWriter('reward_matrix_only_action_2.xlsx') as excel_file:
        p1_reward_matrix.to_excel(excel_file, sheet_name='p1', index=True, header=True)
        p2_reward_matrix.to_excel(excel_file, sheet_name='p2', index=True, header=True)

In [11]:
def calculate_pure_strategy_nash_equilibria(payoffs_row, payoffs_col):
    game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)
    result = pygambit.nash.enumpure_solve(game)
    return result.equilibria

In [12]:
def process_and_print_nash_equilibria_results(nash_equlibria_result, strategy_set_p1, strategy_set_p2):

    number_of_nash_equilbria = len(nash_equlibria_result)
    print("Total number of Nash ", number_of_nash_equilbria)

    '''better readable format of the strategies'''
    strategies_p1_array = np.array(list(strategy_set_p1.keys())) #just the list of strategies
    strategies_p2_array = np.array(list(strategy_set_p2.keys()))

    for eq in nash_equlibria_result:                       # each equilibrium profile
        strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]    #converting mixed strategis to float

        p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool) #boolean conversion - for strategy in pure nash probability =  1, for strategy not in pure nash, probability = 0
        p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
        # float_profile = np.array() # iterate over (strategy, prob

        p1_nash_strategy = strategies_p1_array[p1_active_strategy]
        p2_nash_strategy =  strategies_p2_array[p2_active_strategy]

        
        print('P1:' , p1_nash_strategy)   #will only print the strategy found in nash
        print('P2:' , p2_nash_strategy)

        print('------')


'''


strategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))
strategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))

print(strategy_set_p1_only_action_array)

for eq in result.equilibria:                       # each equilibrium profile
    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]

    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)
    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
    # float_profile = np.array() # iterate over (strategy, prob

    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])
    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])

    print('------')




'''

"\n\n\nstrategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))\nstrategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))\n\nprint(strategy_set_p1_only_action_array)\n\nfor eq in result.equilibria:                       # each equilibrium profile\n    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]\n\n    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)\n    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)\n    # float_profile = np.array() # iterate over (strategy, prob\n\n    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])\n    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])\n\n    print('------')\n\n\n\n\n"

In [13]:
def metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode):
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

    
            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy_obs(policy, mode)
            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix



def metagame_reward_matrix_state_ecopg(strategy_set_p1, strategy_set_p2, mode):
    '''Vi estimate using vis and Ps instead of Vio and obsdist'''
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

            # print(p1_strategy, p2_strategy, "strat p1, p2")

            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy_state_mae_ecopg(policy, mode)

            # print(value_p1, value_p2 ,"value p1, p2")
            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix



def metagame_reward_matrix_state_test(strategy_set_p1, strategy_set_p2, mode, mae):
    '''Vi estimate using vis and Ps instead of Vio and obsdist'''
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

            # print(p1_strategy, p2_strategy, "strat p1, p2")


            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy_state_test(policy, mode, mae)

            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix

In [14]:
def get_pure_nash_equilibria_all_deterministic_strategies_mode(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)
    p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_state_ecopg(strategy_set_p1, strategy_set_p2, mode)
    # p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode)

    # print("p1: \n", p1_reward_matrix, "\n p2: \n", p2_reward_matrix)
    
    nash_equlibrium_result = calculate_pure_strategy_nash_equilibria(p1_reward_matrix, p2_reward_matrix)
    process_and_print_nash_equilibria_results(nash_equlibrium_result, strategy_set_p1, strategy_set_p2)

def get_pure_nash_equilibria_all_deterministic_strategies_mode_obs(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)
    # p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_state_ecopg(strategy_set_p1, strategy_set_p2, mode)
    p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode)

    # print("p1: \n", p1_reward_matrix, "\n p2: \n", p2_reward_matrix)
    
    nash_equlibrium_result = calculate_pure_strategy_nash_equilibria(p1_reward_matrix, p2_reward_matrix)
    process_and_print_nash_equilibria_results(nash_equlibrium_result, strategy_set_p1, strategy_set_p2)

def get_pure_nash_equilibria_all_deterministic_strategies_mode_test(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)
    p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_state_test(strategy_set_p1, strategy_set_p2, mode)
    # p1_reward_matrix, p2_reward_matrix = metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode)

    # print("p1: \n", p1_reward_matrix, "\n p2: \n", p2_reward_matrix)
    
    nash_equlibrium_result = calculate_pure_strategy_nash_equilibria(p1_reward_matrix, p2_reward_matrix)
    process_and_print_nash_equilibria_results(nash_equlibrium_result, strategy_set_p1, strategy_set_p2)

In [15]:
for mode in ['none', 'ecological', 'social', 'complete']:
    print(f"===  mode: {mode} ===")
    get_pure_nash_equilibria_all_deterministic_strategies_mode_obs(mode)


===  mode: none ===
Total number of Nash  1
P1: ['[0.99]']
P2: ['[0.99]']
------
===  mode: ecological ===
Total number of Nash  2
P1: ['[0.99]']
P2: ['[0.99]']
------
P1: ['[0.01]']
P2: ['[0.01]']
------
===  mode: social ===


KeyboardInterrupt: 

###Nash Results


===  mode: complete ===
Total number of Nash  4
P1: ['[0.99, 0.99, 0.99, 0.99]']
P2: ['[0.99, 0.99, 0.99, 0.99]']
------
P1: ['[0.99, 0.01, 0.01, 0.99]']
P2: ['[0.99, 0.01, 0.01, 0.99]']
------
P1: ['[0.01, 0.01, 0.01, 0.99]']
P2: ['[0.01, 0.01, 0.01, 0.99]']
------
P1: ['[0.01, 0.01, 0.01, 0.01]']
P2: ['[0.01, 0.01, 0.01, 0.01]']
------
===  mode: ecological ===
Total number of Nash  2
P1: ['[0.99]']
P2: ['[0.99]']
------
P1: ['[0.01]']
P2: ['[0.01]']
------
===  mode: social ===
Total number of Nash  3
P1: ['[0.99, 0.99, 0.99, 0.99]']
P2: ['[0.99, 0.99, 0.99, 0.99]']
------
P1: ['[0.99, 0.01, 0.01, 0.99]']
P2: ['[0.99, 0.01, 0.01, 0.99]']
------
P1: ['[0.01, 0.01, 0.01, 0.99]']
P2: ['[0.01, 0.01, 0.01, 0.99]']
------
===  mode: none ===
Total number of Nash  1
P1: ['[0.99]']
P2: ['[0.99]']
------

Qvalue

In [ ]:
'''Memo1pd testing for understanding '''



mode = 'social'

env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))

strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)


mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)

p1_reward_matrix_only_state, p2_reward_matrix_only_state = metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode,mae_socdi)

payoffs_row = p1_reward_matrix_only_state
payoffs_col  = p2_reward_matrix_only_state

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


result = pygambit.nash.enumpure_solve(game)



process_and_print_nash_equilibria_results_test(result.equilibria, strategy_set_p1, strategy_set_p2, mode, mae_socdi)

TypeError: metagame_reward_matrix_obs_ecopg() takes 3 positional arguments but 4 were given

Reward prediction error

In [ ]:
def RPE_stability_check(policy, mae):

    prob_of_coop = policy[:, :, 0]
    # print(prob_of_coop)

    reward_prediction_error = mae.RPEisa(policy)
    # print(reward_prediction_error)
    argmin_Q = np.argmin(reward_prediction_error, axis = 2) #if erro i s -.5 theb the poliviy id 
    # print(argmin_Q)

    stability = np.all(prob_of_coop == argmin_Q)
    
    return stability



def check_policy_for_stability_RPE_all_socdi():
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players('social')
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)

    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    results = list(map(RPE_stability_check, policies))
    print(results)
    # print("==", mode, "===")
    print(policies[results])


check_policy_for_stability_RPE_all_socdi()

TypeError: RPE_stability_check() missing 1 required positional argument: 'mae'